In [1]:
import mmcv
from mmcv import Config
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.models import build_detector
from mmdet.apis import single_gpu_test
from mmcv.runner import load_checkpoint
import os
from mmcv.parallel import MMDataParallel
import pandas as pd
from pandas import DataFrame
from pycocotools.coco import COCO
import numpy as np

In [2]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
cfg = Config.fromfile('/opt/ml/Pstage/level2-object-detection-level2-cv-12/baseline/mmdetection/work_dirs/cascade_rcnn_convnext2/0/cascade_rcnn_convnext2.py')

root='/opt/ml/detection/dataset/'

epoch = 'latest'

# dataset config 수정
cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json'
cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize
cfg.data.test.test_mode = True

cfg.data.samples_per_gpu = 4

cfg.seed=2021
cfg.gpu_ids = [1]
cfg.work_dir = '/opt/ml/Pstage/level2-object-detection-level2-cv-12/baseline/mmdetection/work_dirs/cascade_rcnn_convnext2/0'

#cfg.model.roi_head.bbox_head.num_classes = 10

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.model.train_cfg = None

In [3]:
# build dataset & dataloader
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [4]:
# checkpoint path
checkpoint_path = '/opt/ml/Pstage/level2-object-detection-level2-cv-12/baseline/mmdetection/work_dirs/cascade_rcnn_convnext2/0/best_bbox_mAP_50_epoch_11.pth'

model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg')) # build detector
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu') # ckpt load

model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

**************************생성생성생성********************************
load checkpoint from local path: /opt/ml/Pstage/level2-object-detection-level2-cv-12/baseline/mmdetection/work_dirs/cascade_rcnn_convnext2/0/best_bbox_mAP_50_epoch_11.pth


In [5]:
output = single_gpu_test(model, data_loader, show_score_thr=0.05) # output 계산

[>>>>>>>>>>>>>>>>>>>>>>>>>>] 4871/4871, 14.7 task/s, elapsed: 332s, ETA:     0s

In [6]:
# submission 양식에 맞게 output 후처리
prediction_strings = []
file_names = []
coco = COCO(cfg.data.test.ann_file)
img_ids = coco.getImgIds()

class_num = 10
for i, out in enumerate(output):
    prediction_string = ''
    image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
    for j in range(class_num):
        for o in out[j]:
            prediction_string += str(j) + ' ' + str(o[4]) + ' ' + str(o[0]) + ' ' + str(o[1]) + ' ' + str(
                o[2]) + ' ' + str(o[3]) + ' '
        
    prediction_strings.append(prediction_string)
    file_names.append(image_info['file_name'])


submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(os.path.join(cfg.work_dir, f'submission_{epoch}.csv'), index=None)
submission.head()

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


,PredictionString,image_id
0,0 0.21860133 465.5523 585.9109 578.0767 680.99...,test/0000.jpg
1,0 0.03409556 349.7883 241.96297 756.9347 695.3...,test/0001.jpg
2,0 0.06924729 776.36414 401.17358 1024.0 1024.0...,test/0002.jpg
3,0 0.007647405 0.0 2.0380554 1024.0 614.15515 0...,test/0003.jpg
4,0 0.6706466 421.94656 403.60217 662.9496 571.1...,test/0004.jpg


In [7]:
submission['PredictionString'][487]

'0 0.7721302 331.13107 338.77728 660.6915 538.2597 0 0.0025083225 342.74158 431.9501 569.29926 541.5226 0 0.00060478266 275.13745 334.2583 593.1977 384.40033 0 0.0005648735 492.22577 386.94547 574.0705 467.058 0 0.00050071575 347.0727 349.65973 582.36914 477.71124 0 0.0004157596 312.83197 341.2868 473.56586 382.1994 0 0.0004054794 454.92444 393.1753 583.20044 432.96484 0 0.00021497045 484.09473 379.3868 585.08496 417.37234 0 0.00018503073 483.19342 372.52948 602.3172 486.41748 0 0.00017005585 319.17776 329.52277 635.8784 423.25262 0 0.00015141015 444.19217 403.15167 565.6528 445.64514 0 0.00013295011 338.80258 342.18597 470.67227 475.589 0 8.683583e-05 216.8084 304.05002 547.859 519.56586 0 8.215403e-05 345.33395 287.5272 875.1793 449.5582 0 7.9905796e-05 217.24437 8.670563 669.43317 555.48425 0 7.708343e-05 354.78357 383.4372 511.2329 447.16522 0 7.075919e-05 424.02222 388.61273 580.07587 417.826 0 5.7611083e-05 174.43195 301.4775 671.3286 397.85446 0 5.3308922e-05 353.8379 266.73907 